# Classification Pneumonie

## Import

In [ ]:
import os

import numpy as np
import pandas as pd 
import imageio
import random as rdm

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageOps
import scipy.ndimage as ndi

import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout, Activation, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing import image
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils import plot_model

import itertools  
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve, roc_curve, accuracy_score, confusion_matrix, precision_score, recall_score

## Define paths

In [ ]:
dirname = 'chest_xray/'
train_path = os.path.join(dirname, 'chest_xray/train')
train_nrml_pth = os.path.join(train_path, 'NORMAL')
train_pnm_pth = os.path.join(train_path, 'PNEUMONIA')
test_path = os.path.join(dirname, 'chest_xray/test')
test_nrml_pth = os.path.join(test_path, 'NORMAL')
test_pnm_pth = os.path.join(test_path, 'PNEUMONIA')
val_path = os.path.join(dirname, 'chest_xray/val')
val_nrml_pth = os.path.join(val_path, 'NORMAL')
val_pnm_pth = os.path.join(val_path, 'PNEUMONIA')

## Plotting image

In [ ]:
def plot_imgs(item_dir, num_imgs=25):
    all_item_dirs = os.listdir(item_dir)
    item_files = [os.path.join(item_dir, file) for file in all_item_dirs][:num_imgs]

    plt.figure(figsize=(10, 10))
    for idx, img_path in enumerate(item_files):
        plt.subplot(5, 5, idx+1)

        img = plt.imread(img_path)
        #print(img.shape)
        plt.imshow(img, cmap='gray')

    plt.tight_layout()

### Healthy people

In [ ]:
plot_imgs(train_nrml_pth, 5)

### People with pnemonia

In [ ]:
plot_imgs(train_pnm_pth, 5)

## Création du modèle

In [ ]:
img_size_h = 300
img_size_w = 300

input_shape = (img_size_h, img_size_w, 3) 

In [ ]:
# Create VGG16 Model with Keras library
from keras.applications import VGG16

def build_model(backbone, lr = 0.0001):
    model = Sequential()
    model.add(backbone)
    
    model.add(Flatten())
    
    model.add(Dense(1024, activation="relu"))
    model.add(Dropout(0.5))
    
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.5))
    
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.5))
    
    model.add(Dense(1,activation="sigmoid"))
    
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(lr=lr, decay=1e-5),
        metrics=['acc']
    )
    return model

vgg16 = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=input_shape
)

model = build_model(vgg16 ,lr = 1e-4)
model.summary()

## Data processing / Keras Data Generator

In [ ]:
#augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

#augmentation configuration we will use for validation / testing set
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
batch_size = 16
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_size_h, img_size_w),
    batch_size=batch_size,
    class_mode='binary',
    #subset='training',
    color_mode='rgb',
    shuffle=True,
    seed = 1)

validation_generator = test_datagen.flow_from_directory(
    val_path,
    target_size=(img_size_h, img_size_w),
    batch_size=batch_size,
    class_mode='binary',
    #subset='validation',
    color_mode='rgb',
    shuffle=True,
    seed = 1)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_size_h, img_size_w),
    batch_size=batch_size,
    class_mode='binary',
    color_mode='rgb',
    shuffle=True,
    seed = 1)

## Hyperparamètres and Training

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.001)

early_stopping_monitor = EarlyStopping(patience = 4, monitor = "val_acc", mode="max", verbose = 2)

callback = [learning_rate_reduction, early_stopping_monitor]

In [ ]:
history = model.fit_generator(
    train_generator,
    epochs = 5,
    validation_data=validation_generator,
    callbacks = callback,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_steps = validation_generator.samples // batch_size,
    )

### Sauvegarde du modèle

In [ ]:
# model save
model.save_weights("vgg16-example-pneumonia.h5")

In [ ]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/vgg16') 

In [ ]:
#model = keras.models.load_model('saved_model/my_modelV2')

## Plots et prédiction

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
def get_datagenerator(generator):
    data_list = []
    data_class = []
    batch_index = 0

    while batch_index <= generator.batch_index:
        data = generator.next()
        data_list.append(data[0])
        data_class.append(data[1])
        batch_index = batch_index + 1

    # now, data_array is the numeric data of whole images
    train_data = np.concatenate(data_list)
    train_labels = np.concatenate(data_class)
    
    return train_data, train_labels

In [ ]:
test_data, test_labels = get_datagenerator(test_generator)

# Evaluation on test dataset
test_loss, test_score = model.evaluate(test_data, test_labels, batch_size = batch_size)
print("Loss on test set: ", test_loss)
print("Accuracy on test set: ", test_score)

In [ ]:
# Get predictions
preds = model.predict(test_data, batch_size = batch_size)
preds = np.concatenate(preds)

In [ ]:
precisions, recalls, thresholds = precision_recall_curve(test_labels, preds)
fpr, tpr, thresholds2 = roc_curve(test_labels, preds)

In [ ]:
def plot_precision_recall(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], 'b--')
    plt.plot(thresholds, recalls[:-1], 'g-')
    plt.title('Precision vs. Recall')
    plt.xlabel('Thresholds')
    plt.legend(['Precision', 'Recall'], loc='best')
    plt.show()

def plot_roc(fpr, tpr):
    plt.plot(fpr, tpr)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title('FPR (False Positive rate) vs TPR (True Positive Rate)')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate (Recall)')
    plt.show()
    
plot_precision_recall(precisions, recalls, thresholds)
plot_roc(fpr, tpr)

In [ ]:
# Plot non-normalized confusion matrix
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=0)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(test_labels, preds)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
class_names = test_generator.class_indices.keys()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

## Test du modèle sur quelques exemples

In [ ]:
def predict_illness(image_path):
    imge = plt.imread(image_path)
    plt.figure()

    img = image.load_img(image_path, target_size=(img_size_h, img_size_w), grayscale = "True")
    x = image.img_to_array(img) 
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])

    classes = model.predict_classes(images, batch_size=10)
    if classes[0][0] == 0:
        textstr = "Prediction: Healthy"
    else:
        textstr = "Prediction: Pneumonia"

    if ('chest_xray/chest_xray/test/NORMAL' in image_path):
        texttruth = "Truth: Healthy"
    else :
        texttruth = "Truth: Pneumonia"
        
    plt.gcf().text(0.9, 0.4, textstr, fontsize=20)
    plt.gcf().text(0.9, 0.6, texttruth, fontsize=20)
    plt.imshow(img, cmap='gray')

In [ ]:
k = 5
sampling_normal = rdm.sample(os.listdir(test_nrml_pth), k=k)
sampling_pnm = rdm.sample(os.listdir(test_pnm_pth), k=k)

for i in range(0,k):
    
    img_test_nrml = os.path.join(test_nrml_pth, sampling_normal[i])
    img_test_ill = os.path.join(test_pnm_pth, sampling_pnm[i])
    
    imge = predict_illness(img_test_nrml)
    imge = predict_illness(img_test_ill)